# TensorFlow Hub - Text Classification

# Importing the libraries

In [31]:
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
print("TensorFlow version:", tf.__version__)
print("TensorFlow Hub version:", hub.__version__)

TensorFlow version: 2.15.0
TensorFlow Hub version: 0.16.1


# Dataset

 ####  Extracted from imbd: 0 - negative, 1 - positive

In [32]:
train_data, test_data = tfds.load(name = 'imdb_reviews', split =['train', 'test'], batch_size=-1, as_supervised=True)
X_train, y_train = tfds.as_numpy(train_data)
X_test, y_test = tfds.as_numpy(test_data)
# name = "imbd_reviews" --> imbd is a famous website that presents a lot of moive reviews
# batch_size=-1 --> to not consider the data set in batches
# as_supervised=True --> because we want to get the texts and corresponding labels

In [33]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(25000,)
(25000,)
(25000,)
(25000,)


In [34]:
X_train[0:2]

array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The plot 

In [35]:
y_train[0:2]
# which means they are negative reviews

array([0, 0])

In [36]:
X_test[0:2]

array([b"There are films that make careers. For George Romero, it was NIGHT OF THE LIVING DEAD; for Kevin Smith, CLERKS; for Robert Rodriguez, EL MARIACHI. Add to that list Onur Tukel's absolutely amazing DING-A-LING-LESS. Flawless film-making, and as assured and as professional as any of the aforementioned movies. I haven't laughed this hard since I saw THE FULL MONTY. (And, even then, I don't think I laughed quite this hard... So to speak.) Tukel's talent is considerable: DING-A-LING-LESS is so chock full of double entendres that one would have to sit down with a copy of this script and do a line-by-line examination of it to fully appreciate the, uh, breadth and width of it. Every shot is beautifully composed (a clear sign of a sure-handed director), and the performances all around are solid (there's none of the over-the-top scenery chewing one might've expected from a film like this). DING-A-LING-LESS is a film whose time has come.",
       b"A blackly comic tale of a down-trodden p

In [37]:
y_test[0:2]
# which means they are both positive

array([1, 1])

In [38]:
np.unique(y_train, return_counts = True)

(array([0, 1]), array([12500, 12500]))

In [39]:
np.unique(y_test, return_counts = True)

(array([0, 1]), array([12500, 12500]))

# Building and training the neural network

In [40]:
# a custom model
# embedding --> it will convert text to numbers
model_path = 'https://tfhub.dev/google/nnlm-en-dim50/2'

In [41]:
embedding_layer = hub.KerasLayer(model_path, input_shape = [], dtype = tf.string, trainable = True)


In [42]:
embedding_layer(X_train[0:2])

<tf.Tensor: shape=(2, 50), dtype=float32, numpy=
array([[ 0.5423194 , -0.01190171,  0.06337537,  0.0686297 , -0.16776839,
        -0.10581177,  0.168653  , -0.04998823, -0.31148052,  0.07910344,
         0.15442258,  0.01488661,  0.03930155,  0.19772716, -0.12215477,
        -0.04120982, -0.27041087, -0.21922147,  0.26517656, -0.80739075,
         0.25833526, -0.31004202,  0.2868321 ,  0.19433866, -0.29036498,
         0.0386285 , -0.78444123, -0.04793238,  0.41102988, -0.36388886,
        -0.58034706,  0.30269453,  0.36308962, -0.15227163, -0.4439151 ,
         0.19462997,  0.19528405,  0.05666233,  0.2890704 , -0.28468323,
        -0.00531206,  0.0571938 , -0.3201319 , -0.04418665, -0.08550781,
        -0.55847436, -0.2333639 , -0.20782956, -0.03543065, -0.17533456],
       [ 0.56338924, -0.12339553, -0.10862677,  0.7753425 , -0.07667087,
        -0.15752274,  0.01872334, -0.08169781, -0.3521876 ,  0.46373403,
        -0.08492758,  0.07166861, -0.00670818,  0.12686071, -0.19326551,
 

In [43]:
model = tf.keras.Sequential()
model.add(embedding_layer)
model.add(tf.keras.layers.Dense(units = 16, activation = 'relu'))
model.add(tf.keras.layers.Dense(units = 1))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_1 (KerasLayer)  (None, 50)                48190600  
                                                                 
 dense_2 (Dense)             (None, 16)                816       
                                                                 
 dense_3 (Dense)             (None, 1)                 17        
                                                                 
Total params: 48191433 (183.84 MB)
Trainable params: 48191433 (183.84 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [44]:
model.compile(optimizer = 'adam', loss = tf.losses.BinaryCrossentropy(from_logits = True), metrics=['accuracy'])

In [45]:
model.fit(X_train, y_train, epochs = 20, batch_size = 512, verbose = 1) # verbose -> to see the results while training

Epoch 1/20
49/49 [==============================] - 16s 269ms/step - loss: 0.6480 - accuracy: 0.5356
Epoch 2/20
49/49 [==============================] - 7s 142ms/step - loss: 0.4679 - accuracy: 0.7731
Epoch 3/20
49/49 [==============================] - 8s 155ms/step - loss: 0.3023 - accuracy: 0.8811
Epoch 4/20
49/49 [==============================] - 6s 120ms/step - loss: 0.2106 - accuracy: 0.9203
Epoch 5/20
49/49 [==============================] - 7s 138ms/step - loss: 0.1505 - accuracy: 0.9480
Epoch 6/20
49/49 [==============================] - 7s 136ms/step - loss: 0.1071 - accuracy: 0.9665
Epoch 7/20
49/49 [==============================] - 6s 129ms/step - loss: 0.0752 - accuracy: 0.9795
Epoch 8/20
49/49 [==============================] - 7s 145ms/step - loss: 0.0526 - accuracy: 0.9883
Epoch 9/20
49/49 [==============================] - 6s 125ms/step - loss: 0.0364 - accuracy: 0.9939
Epoch 10/20
49/49 [==============================] - 6s 131ms/step - loss: 0.0258 - accuracy: 0.997

In [46]:
results = model.evaluate(X_test, y_test)
print(results)

782/782 [==============================] - 4s 5ms/step - loss: 0.5774 - accuracy: 0.8497
[0.5773702263832092, 0.8496800065040588]


# Predictions

In [47]:
X_test[0:6]

array([b"There are films that make careers. For George Romero, it was NIGHT OF THE LIVING DEAD; for Kevin Smith, CLERKS; for Robert Rodriguez, EL MARIACHI. Add to that list Onur Tukel's absolutely amazing DING-A-LING-LESS. Flawless film-making, and as assured and as professional as any of the aforementioned movies. I haven't laughed this hard since I saw THE FULL MONTY. (And, even then, I don't think I laughed quite this hard... So to speak.) Tukel's talent is considerable: DING-A-LING-LESS is so chock full of double entendres that one would have to sit down with a copy of this script and do a line-by-line examination of it to fully appreciate the, uh, breadth and width of it. Every shot is beautifully composed (a clear sign of a sure-handed director), and the performances all around are solid (there's none of the over-the-top scenery chewing one might've expected from a film like this). DING-A-LING-LESS is a film whose time has come.",
       b"A blackly comic tale of a down-trodden p

In [48]:
y_test[0:6]

array([1, 1, 0, 0, 1, 1])

In [49]:
predictions = model.predict(X_test[0:6])

1/1 [==============================] - 0s 62ms/step


In [50]:
predictions

array([[  2.5174441],
       [  0.8525399],
       [ -4.4115205],
       [-10.927716 ],
       [  8.908579 ],
       [ 11.231491 ]], dtype=float32)

In [51]:
predictions = tf.nn.sigmoid(predictions).numpy() # sigmoid --> will return numbers from 0 to 1 probabilities.
predictions

array([[9.2535573e-01],
       [7.0109969e-01],
       [1.1991177e-02],
       [1.7953344e-05],
       [9.9986482e-01],
       [9.9998677e-01]], dtype=float32)

In [52]:
y_test[0:6] # 0 - negative, 1 - positive

array([1, 1, 0, 0, 1, 1])

In [53]:
predictions = (predictions >= 0.5)
predictions

array([[ True],
       [ True],
       [False],
       [False],
       [ True],
       [ True]])